##Importing libraries

In [0]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## Loading the data file

In [16]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
root_path = 'gdrive/My Drive/'
merged_features = pd.read_csv(root_path+'merged_features.csv')
merged_features.head()

,Unnamed: 0,YEAR,MONTH,LATITUDE,LONGITUDE,NOM_ARROND,NON_FIRE_EVENT_COUNT,FIRE_EVENT_COUNT,FIRE_EVENT,Résidentiel_BUILDING_COUNT,Résidentiel_STORY_COUNT,Résidentiel_UNIT_COUNT,Résidentiel_AGE_AVG,Résidentiel_LAND_AREA_TOTAL,Résidentiel_BUILDING_AREA_TOTAL,Commercial_BUILDING_COUNT,Commercial_STORY_COUNT,Commercial_UNIT_COUNT,Commercial_AGE_AVG,Commercial_LAND_AREA_TOTAL,Commercial_BUILDING_AREA_TOTAL,Terrain vague_BUILDING_COUNT,Terrain vague_LAND_AREA_TOTAL,Terrain vague_BUILDING_AREA_TOTAL,Ruelle_BUILDING_COUNT,Ruelle_LAND_AREA_TOTAL,Industriel_BUILDING_COUNT,Industriel_STORY_COUNT,Industriel_UNIT_COUNT,Industriel_AGE_AVG,Industriel_LAND_AREA_TOTAL,Industriel_BUILDING_AREA_TOTAL,Transport_BUILDING_COUNT,Transport_STORY_COUNT,Transport_UNIT_COUNT,Transport_AGE_AVG,Transport_LAND_AREA_TOTAL,Transport_BUILDING_AREA_TOTAL,Parc_BUILDING_COUNT,Parc_STORY_COUNT,...,Immeuble inutilisé_BUILDING_COUNT,Immeuble inutilisé_STORY_COUNT,Immeuble inutilisé_AGE_AVG,Immeuble inutilisé_LAND_AREA_TOTAL,Immeuble inutilisé_BUILDING_AREA_TOTAL,Éducation_BUILDING_COUNT,Éducation_STORY_COUNT,Éducation_UNIT_COUNT,Éducation_AGE_AVG,Éducation_LAND_AREA_TOTAL,Éducation_BUILDING_AREA_TOTAL,Loisirs_BUILDING_COUNT,Loisirs_STORY_COUNT,Loisirs_UNIT_COUNT,Loisirs_AGE_AVG,Loisirs_LAND_AREA_TOTAL,Loisirs_BUILDING_AREA_TOTAL,Santé_BUILDING_COUNT,Santé_STORY_COUNT,Santé_UNIT_COUNT,Santé_AGE_AVG,Santé_LAND_AREA_TOTAL,Santé_BUILDING_AREA_TOTAL,Sécurité publique_BUILDING_COUNT,Sécurité publique_STORY_COUNT,Sécurité publique_AGE_AVG,Sécurité publique_LAND_AREA_TOTAL,Sécurité publique_BUILDING_AREA_TOTAL,Restaurant_BUILDING_COUNT,Restaurant_STORY_COUNT,Restaurant_UNIT_COUNT,Restaurant_AGE_AVG,Restaurant_LAND_AREA_TOTAL,Restaurant_BUILDING_AREA_TOTAL,Étudiants_BUILDING_COUNT,Étudiants_STORY_COUNT,Étudiants_UNIT_COUNT,Étudiants_AGE_AVG,Étudiants_LAND_AREA_TOTAL,Étudiants_BUILDING_AREA_TOTAL
0,0,2005,1,45.62,-73.62,Montréal-Nord,33,8,1,12424,22912.0,38621.0,41.689070,4774185.0,3375618.0,373,320.0,10.0,36.528150,1157943.0,429020.0,266,501938.0,8471.0,19,886.0,73,44.0,0.0,30.808219,280703.0,101218.0,258,134.0,3.0,36.116279,368366.0,96594.0,19,1.0,...,5,1.0,27.800000,3250.0,621.0,46,57.0,9.0,43.130435,355991.0,210481.0,3,0.0,0.0,25.666667,63256.0,6022.0,2,1.0,0.0,34.00,12369.0,12007.0,2,0.0,15.000000,28972.0,2549.0,6,7.0,0.0,18.000000,12172.0,1517.0,1,4.0,5.0,9.000000,5220.0,0.0
1,1,2005,1,45.49,-73.58,Ville-Marie / Parc Jean-Drapeau / Centre-Sud,35,24,1,21903,26711.0,54682.0,51.711409,2537161.0,16476280.0,2373,3348.0,297.0,55.383059,1706791.0,7617611.0,2009,860141.0,89618.0,2,1198.0,41,55.0,6.0,53.853659,846167.0,167834.0,198,50.0,14.0,19.530303,2464043.0,56181.0,65,6.0,...,10,33.0,80.300000,59627.0,14833.0,88,170.0,42.0,55.625000,511539.0,910873.0,40,26.0,4.0,56.625000,539817.0,137906.0,24,13.0,0.0,59.25,276251.0,285261.0,6,8.0,61.833333,24472.0,15553.0,46,43.0,1.0,65.695652,10946.0,89167.0,13,70.0,26.0,62.076923,48131.0,125131.0
2,2,2005,1,45.48,-73.69,St-Laurent,2,2,1,18856,23343.0,35527.0,29.361423,5763462.0,9377079.0,726,723.0,5.0,29.013774,8090310.0,2719256.0,715,3894999.0,39219.0,18,5182.0,240,121.0,0.0,22.375000,3692195.0,1254622.0,579,423.0,0.0,26.350604,9890935.0,1093602.0,48,2.0,...,0,0.0,0.000000,0.0,0.0,59,51.0,2.0,43.559322,741242.0,186990.0,6,8.0,2.0,49.333333,80889.0,35399.0,2,3.0,0.0,37.00,4667.0,844.0,3,0.0,6.333333,29704.0,1789.0,8,8.0,0.0,23.125000,21994.0,1516.0,3,7.0,165.0,39.000000,4991.0,102.0
3,3,2005,1,45.54,-73.55,Mercier / Hochelaga-Maisonneuve,8,3,1,25257,44836.0,66187.0,49.637012,6780004.0,9244519.0,368,392.0,26.0,45.290761,1881853.0,1633321.0,755,1540265.0,25553.0,9,10843.0,57,67.0,0.0,45.894737,3197240.0,566205.0,95,41.0,10.0,27.863158,964932.0,66015.0,39,7.0,...,3,2.0,40.666667,16564.0,848.0,80,58.0,2.0,40.162500,753976.0,261331.0,15,12.0,18.0,44.000000,171880.0,28484.0,10,5.0,6.0,42.90,330657.0,131033.0,6,4.0,57.166667,860673.0,221951.0,26,25.0,0.0,35.923077,41118.0,8020.0,0,0.0,0.0,0.000000,0.0,0.0
4,4,2005,1,45.61,-73.63,Montréal-Nord,9,

## Final cleaning: Factorizing string data

In [18]:
merged_features['NOM_ARROND'], unique_arronds = pd.factorize(merged_features['NOM_ARROND'])
unique_arronds

Index(['Montréal-Nord', 'Ville-Marie / Parc Jean-Drapeau / Centre-Sud',
       'St-Laurent', 'Mercier / Hochelaga-Maisonneuve', 'Anjou', 'Westmount',
       'Plateau Mont-Royal', 'Rivière-des-Prairies / Pointe-aux-Trembles',
       'Ahuntsic / Cartierville', 'St-Léonard', 'Rosemont / Petite-Patrie',
       'Ile-Bizard / Ste-Geneviève / Ste-A-de-B',
       'Villeray / St-Michel / Parc Extension', 'Verdun', 'Sud-Ouest',
       'Lasalle', 'Côte-des-Neiges / Notre-Dame-de-Grâce', 'Mont-Royal',
       'Dollard-des-Ormeaux / Roxboro', 'Pierrefonds / Roxboro / Senneville',
       'Pointe-Claire', 'Côte St-Luc / Hampstead / Mtl-Ouest',
       'Dorval / Ile Dorval', 'Beaconsfield / Baie d'Urfé', 'Lachine',
       'Outremont', 'Kirkland', 'Ste-Anne-de-Bellevue', 'Montréal-Est'],
      dtype='object')

In [19]:
merged_features.head()

,Unnamed: 0,YEAR,MONTH,LATITUDE,LONGITUDE,NOM_ARROND,NON_FIRE_EVENT_COUNT,FIRE_EVENT_COUNT,FIRE_EVENT,Résidentiel_BUILDING_COUNT,Résidentiel_STORY_COUNT,Résidentiel_UNIT_COUNT,Résidentiel_AGE_AVG,Résidentiel_LAND_AREA_TOTAL,Résidentiel_BUILDING_AREA_TOTAL,Commercial_BUILDING_COUNT,Commercial_STORY_COUNT,Commercial_UNIT_COUNT,Commercial_AGE_AVG,Commercial_LAND_AREA_TOTAL,Commercial_BUILDING_AREA_TOTAL,Terrain vague_BUILDING_COUNT,Terrain vague_LAND_AREA_TOTAL,Terrain vague_BUILDING_AREA_TOTAL,Ruelle_BUILDING_COUNT,Ruelle_LAND_AREA_TOTAL,Industriel_BUILDING_COUNT,Industriel_STORY_COUNT,Industriel_UNIT_COUNT,Industriel_AGE_AVG,Industriel_LAND_AREA_TOTAL,Industriel_BUILDING_AREA_TOTAL,Transport_BUILDING_COUNT,Transport_STORY_COUNT,Transport_UNIT_COUNT,Transport_AGE_AVG,Transport_LAND_AREA_TOTAL,Transport_BUILDING_AREA_TOTAL,Parc_BUILDING_COUNT,Parc_STORY_COUNT,...,Immeuble inutilisé_BUILDING_COUNT,Immeuble inutilisé_STORY_COUNT,Immeuble inutilisé_AGE_AVG,Immeuble inutilisé_LAND_AREA_TOTAL,Immeuble inutilisé_BUILDING_AREA_TOTAL,Éducation_BUILDING_COUNT,Éducation_STORY_COUNT,Éducation_UNIT_COUNT,Éducation_AGE_AVG,Éducation_LAND_AREA_TOTAL,Éducation_BUILDING_AREA_TOTAL,Loisirs_BUILDING_COUNT,Loisirs_STORY_COUNT,Loisirs_UNIT_COUNT,Loisirs_AGE_AVG,Loisirs_LAND_AREA_TOTAL,Loisirs_BUILDING_AREA_TOTAL,Santé_BUILDING_COUNT,Santé_STORY_COUNT,Santé_UNIT_COUNT,Santé_AGE_AVG,Santé_LAND_AREA_TOTAL,Santé_BUILDING_AREA_TOTAL,Sécurité publique_BUILDING_COUNT,Sécurité publique_STORY_COUNT,Sécurité publique_AGE_AVG,Sécurité publique_LAND_AREA_TOTAL,Sécurité publique_BUILDING_AREA_TOTAL,Restaurant_BUILDING_COUNT,Restaurant_STORY_COUNT,Restaurant_UNIT_COUNT,Restaurant_AGE_AVG,Restaurant_LAND_AREA_TOTAL,Restaurant_BUILDING_AREA_TOTAL,Étudiants_BUILDING_COUNT,Étudiants_STORY_COUNT,Étudiants_UNIT_COUNT,Étudiants_AGE_AVG,Étudiants_LAND_AREA_TOTAL,Étudiants_BUILDING_AREA_TOTAL
0,0,2005,1,45.62,-73.62,0,33,8,1,12424,22912.0,38621.0,41.689070,4774185.0,3375618.0,373,320.0,10.0,36.528150,1157943.0,429020.0,266,501938.0,8471.0,19,886.0,73,44.0,0.0,30.808219,280703.0,101218.0,258,134.0,3.0,36.116279,368366.0,96594.0,19,1.0,...,5,1.0,27.800000,3250.0,621.0,46,57.0,9.0,43.130435,355991.0,210481.0,3,0.0,0.0,25.666667,63256.0,6022.0,2,1.0,0.0,34.00,12369.0,12007.0,2,0.0,15.000000,28972.0,2549.0,6,7.0,0.0,18.000000,12172.0,1517.0,1,4.0,5.0,9.000000,5220.0,0.0
1,1,2005,1,45.49,-73.58,1,35,24,1,21903,26711.0,54682.0,51.711409,2537161.0,16476280.0,2373,3348.0,297.0,55.383059,1706791.0,7617611.0,2009,860141.0,89618.0,2,1198.0,41,55.0,6.0,53.853659,846167.0,167834.0,198,50.0,14.0,19.530303,2464043.0,56181.0,65,6.0,...,10,33.0,80.300000,59627.0,14833.0,88,170.0,42.0,55.625000,511539.0,910873.0,40,26.0,4.0,56.625000,539817.0,137906.0,24,13.0,0.0,59.25,276251.0,285261.0,6,8.0,61.833333,24472.0,15553.0,46,43.0,1.0,65.695652,10946.0,89167.0,13,70.0,26.0,62.076923,48131.0,125131.0
2,2,2005,1,45.48,-73.69,2,2,2,1,18856,23343.0,35527.0,29.361423,5763462.0,9377079.0,726,723.0,5.0,29.013774,8090310.0,2719256.0,715,3894999.0,39219.0,18,5182.0,240,121.0,0.0,22.375000,3692195.0,1254622.0,579,423.0,0.0,26.350604,9890935.0,1093602.0,48,2.0,...,0,0.0,0.000000,0.0,0.0,59,51.0,2.0,43.559322,741242.0,186990.0,6,8.0,2.0,49.333333,80889.0,35399.0,2,3.0,0.0,37.00,4667.0,844.0,3,0.0,6.333333,29704.0,1789.0,8,8.0,0.0,23.125000,21994.0,1516.0,3,7.0,165.0,39.000000,4991.0,102.0
3,3,2005,1,45.54,-73.55,3,8,3,1,25257,44836.0,66187.0,49.637012,6780004.0,9244519.0,368,392.0,26.0,45.290761,1881853.0,1633321.0,755,1540265.0,25553.0,9,10843.0,57,67.0,0.0,45.894737,3197240.0,566205.0,95,41.0,10.0,27.863158,964932.0,66015.0,39,7.0,...,3,2.0,40.666667,16564.0,848.0,80,58.0,2.0,40.162500,753976.0,261331.0,15,12.0,18.0,44.000000,171880.0,28484.0,10,5.0,6.0,42.90,330657.0,131033.0,6,4.0,57.166667,860673.0,221951.0,26,25.0,0.0,35.923077,41118.0,8020.0,0,0.0,0.0,0.000000,0.0,0.0
4,4,2005,1,45.61,-73.63,0,9,3,1,12424,22912.0,38621.0,41.689070,4774185.0,3375618.0,373,320.0,10.0,36.528150,1157943.0,429020.0,266,50


## Binning the risk of fires

In [20]:
merged_features['FIRE_EVENT_COUNT'].max()

59

In [21]:
# Each line is on a per month basis. So we define the bins as such:
#    1:Low (0 to 4 fire events per month)
#    2:Medium (5 to 19 fire events per month)
#    3:High (more than 20 fire events per month)
labels = [1,2,3]
bins = [-1, 4, 20, merged_features['FIRE_EVENT_COUNT'].max()]
merged_features['FIRE_RISK'] = pd.cut(merged_features['FIRE_EVENT_COUNT'], bins=bins, labels=labels)
merged_features.head()

,Unnamed: 0,YEAR,MONTH,LATITUDE,LONGITUDE,NOM_ARROND,NON_FIRE_EVENT_COUNT,FIRE_EVENT_COUNT,FIRE_EVENT,Résidentiel_BUILDING_COUNT,Résidentiel_STORY_COUNT,Résidentiel_UNIT_COUNT,Résidentiel_AGE_AVG,Résidentiel_LAND_AREA_TOTAL,Résidentiel_BUILDING_AREA_TOTAL,Commercial_BUILDING_COUNT,Commercial_STORY_COUNT,Commercial_UNIT_COUNT,Commercial_AGE_AVG,Commercial_LAND_AREA_TOTAL,Commercial_BUILDING_AREA_TOTAL,Terrain vague_BUILDING_COUNT,Terrain vague_LAND_AREA_TOTAL,Terrain vague_BUILDING_AREA_TOTAL,Ruelle_BUILDING_COUNT,Ruelle_LAND_AREA_TOTAL,Industriel_BUILDING_COUNT,Industriel_STORY_COUNT,Industriel_UNIT_COUNT,Industriel_AGE_AVG,Industriel_LAND_AREA_TOTAL,Industriel_BUILDING_AREA_TOTAL,Transport_BUILDING_COUNT,Transport_STORY_COUNT,Transport_UNIT_COUNT,Transport_AGE_AVG,Transport_LAND_AREA_TOTAL,Transport_BUILDING_AREA_TOTAL,Parc_BUILDING_COUNT,Parc_STORY_COUNT,...,Immeuble inutilisé_STORY_COUNT,Immeuble inutilisé_AGE_AVG,Immeuble inutilisé_LAND_AREA_TOTAL,Immeuble inutilisé_BUILDING_AREA_TOTAL,Éducation_BUILDING_COUNT,Éducation_STORY_COUNT,Éducation_UNIT_COUNT,Éducation_AGE_AVG,Éducation_LAND_AREA_TOTAL,Éducation_BUILDING_AREA_TOTAL,Loisirs_BUILDING_COUNT,Loisirs_STORY_COUNT,Loisirs_UNIT_COUNT,Loisirs_AGE_AVG,Loisirs_LAND_AREA_TOTAL,Loisirs_BUILDING_AREA_TOTAL,Santé_BUILDING_COUNT,Santé_STORY_COUNT,Santé_UNIT_COUNT,Santé_AGE_AVG,Santé_LAND_AREA_TOTAL,Santé_BUILDING_AREA_TOTAL,Sécurité publique_BUILDING_COUNT,Sécurité publique_STORY_COUNT,Sécurité publique_AGE_AVG,Sécurité publique_LAND_AREA_TOTAL,Sécurité publique_BUILDING_AREA_TOTAL,Restaurant_BUILDING_COUNT,Restaurant_STORY_COUNT,Restaurant_UNIT_COUNT,Restaurant_AGE_AVG,Restaurant_LAND_AREA_TOTAL,Restaurant_BUILDING_AREA_TOTAL,Étudiants_BUILDING_COUNT,Étudiants_STORY_COUNT,Étudiants_UNIT_COUNT,Étudiants_AGE_AVG,Étudiants_LAND_AREA_TOTAL,Étudiants_BUILDING_AREA_TOTAL,FIRE_RISK
0,0,2005,1,45.62,-73.62,0,33,8,1,12424,22912.0,38621.0,41.689070,4774185.0,3375618.0,373,320.0,10.0,36.528150,1157943.0,429020.0,266,501938.0,8471.0,19,886.0,73,44.0,0.0,30.808219,280703.0,101218.0,258,134.0,3.0,36.116279,368366.0,96594.0,19,1.0,...,1.0,27.800000,3250.0,621.0,46,57.0,9.0,43.130435,355991.0,210481.0,3,0.0,0.0,25.666667,63256.0,6022.0,2,1.0,0.0,34.00,12369.0,12007.0,2,0.0,15.000000,28972.0,2549.0,6,7.0,0.0,18.000000,12172.0,1517.0,1,4.0,5.0,9.000000,5220.0,0.0,2
1,1,2005,1,45.49,-73.58,1,35,24,1,21903,26711.0,54682.0,51.711409,2537161.0,16476280.0,2373,3348.0,297.0,55.383059,1706791.0,7617611.0,2009,860141.0,89618.0,2,1198.0,41,55.0,6.0,53.853659,846167.0,167834.0,198,50.0,14.0,19.530303,2464043.0,56181.0,65,6.0,...,33.0,80.300000,59627.0,14833.0,88,170.0,42.0,55.625000,511539.0,910873.0,40,26.0,4.0,56.625000,539817.0,137906.0,24,13.0,0.0,59.25,276251.0,285261.0,6,8.0,61.833333,24472.0,15553.0,46,43.0,1.0,65.695652,10946.0,89167.0,13,70.0,26.0,62.076923,48131.0,125131.0,3
2,2,2005,1,45.48,-73.69,2,2,2,1,18856,23343.0,35527.0,29.361423,5763462.0,9377079.0,726,723.0,5.0,29.013774,8090310.0,2719256.0,715,3894999.0,39219.0,18,5182.0,240,121.0,0.0,22.375000,3692195.0,1254622.0,579,423.0,0.0,26.350604,9890935.0,1093602.0,48,2.0,...,0.0,0.000000,0.0,0.0,59,51.0,2.0,43.559322,741242.0,186990.0,6,8.0,2.0,49.333333,80889.0,35399.0,2,3.0,0.0,37.00,4667.0,844.0,3,0.0,6.333333,29704.0,1789.0,8,8.0,0.0,23.125000,21994.0,1516.0,3,7.0,165.0,39.000000,4991.0,102.0,1
3,3,2005,1,45.54,-73.55,3,8,3,1,25257,44836.0,66187.0,49.637012,6780004.0,9244519.0,368,392.0,26.0,45.290761,1881853.0,1633321.0,755,1540265.0,25553.0,9,10843.0,57,67.0,0.0,45.894737,3197240.0,566205.0,95,41.0,10.0,27.863158,964932.0,66015.0,39,7.0,...,2.0,40.666667,16564.0,848.0,80,58.0,2.0,40.162500,753976.0,261331.0,15,12.0,18.0,44.000000,171880.0,28484.0,10,5.0,6.0,42.90,330657.0,131033.0,6,4.0,57.166667,860673.0,221951.0,26,25.0,0.0,35.923077,41118.0,8020.0,0,0.0,0.0,0.000000,0.0,0.0,1
4,4,2005,1,45.61,-73.63,0,9,3,1,12424,22912.0,38621.0,41.689070,4774185.0,3375618.0,373,320.0,10.0,36.528150,1157943.0,429020.0,266,501938.0,8471.0,19,886.0,73

In [22]:
# Making sure all the data was binned correctly
merged_features['FIRE_RISK'].isna().sum()

0

## Removing some undesired columns

In [0]:
# It would require another model model to predict the other features and use them to make a fire risk prediction, so we can't use them for training!
merged_features = merged_features.drop(['Unnamed: 0', 'NON_FIRE_EVENT_COUNT', 'FIRE_EVENT_COUNT', 'FIRE_EVENT'], axis=1)

Neural Network Model using Adabound Optimizer

In [24]:
merged_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95473 entries, 0 to 95472
Data columns (total 93 columns):
YEAR                                      95473 non-null int64
MONTH                                     95473 non-null int64
LATITUDE                                  95473 non-null float64
LONGITUDE                                 95473 non-null float64
NOM_ARROND                                95473 non-null int64
Résidentiel_BUILDING_COUNT                95473 non-null int64
Résidentiel_STORY_COUNT                   95473 non-null float64
Résidentiel_UNIT_COUNT                    95473 non-null float64
Résidentiel_AGE_AVG                       95473 non-null float64
Résidentiel_LAND_AREA_TOTAL               95473 non-null float64
Résidentiel_BUILDING_AREA_TOTAL           95473 non-null float64
Commercial_BUILDING_COUNT                 95473 non-null int64
Commercial_STORY_COUNT                    95473 non-null float64
Commercial_UNIT_COUNT                     95473 non-null

In [0]:
merged_features['FIRE_RISK'] = merged_features['FIRE_RISK'].astype(np.float64)

In [26]:
merged_features.head()

,YEAR,MONTH,LATITUDE,LONGITUDE,NOM_ARROND,Résidentiel_BUILDING_COUNT,Résidentiel_STORY_COUNT,Résidentiel_UNIT_COUNT,Résidentiel_AGE_AVG,Résidentiel_LAND_AREA_TOTAL,Résidentiel_BUILDING_AREA_TOTAL,Commercial_BUILDING_COUNT,Commercial_STORY_COUNT,Commercial_UNIT_COUNT,Commercial_AGE_AVG,Commercial_LAND_AREA_TOTAL,Commercial_BUILDING_AREA_TOTAL,Terrain vague_BUILDING_COUNT,Terrain vague_LAND_AREA_TOTAL,Terrain vague_BUILDING_AREA_TOTAL,Ruelle_BUILDING_COUNT,Ruelle_LAND_AREA_TOTAL,Industriel_BUILDING_COUNT,Industriel_STORY_COUNT,Industriel_UNIT_COUNT,Industriel_AGE_AVG,Industriel_LAND_AREA_TOTAL,Industriel_BUILDING_AREA_TOTAL,Transport_BUILDING_COUNT,Transport_STORY_COUNT,Transport_UNIT_COUNT,Transport_AGE_AVG,Transport_LAND_AREA_TOTAL,Transport_BUILDING_AREA_TOTAL,Parc_BUILDING_COUNT,Parc_STORY_COUNT,Parc_UNIT_COUNT,Parc_AGE_AVG,Parc_LAND_AREA_TOTAL,Parc_BUILDING_AREA_TOTAL,...,Immeuble inutilisé_STORY_COUNT,Immeuble inutilisé_AGE_AVG,Immeuble inutilisé_LAND_AREA_TOTAL,Immeuble inutilisé_BUILDING_AREA_TOTAL,Éducation_BUILDING_COUNT,Éducation_STORY_COUNT,Éducation_UNIT_COUNT,Éducation_AGE_AVG,Éducation_LAND_AREA_TOTAL,Éducation_BUILDING_AREA_TOTAL,Loisirs_BUILDING_COUNT,Loisirs_STORY_COUNT,Loisirs_UNIT_COUNT,Loisirs_AGE_AVG,Loisirs_LAND_AREA_TOTAL,Loisirs_BUILDING_AREA_TOTAL,Santé_BUILDING_COUNT,Santé_STORY_COUNT,Santé_UNIT_COUNT,Santé_AGE_AVG,Santé_LAND_AREA_TOTAL,Santé_BUILDING_AREA_TOTAL,Sécurité publique_BUILDING_COUNT,Sécurité publique_STORY_COUNT,Sécurité publique_AGE_AVG,Sécurité publique_LAND_AREA_TOTAL,Sécurité publique_BUILDING_AREA_TOTAL,Restaurant_BUILDING_COUNT,Restaurant_STORY_COUNT,Restaurant_UNIT_COUNT,Restaurant_AGE_AVG,Restaurant_LAND_AREA_TOTAL,Restaurant_BUILDING_AREA_TOTAL,Étudiants_BUILDING_COUNT,Étudiants_STORY_COUNT,Étudiants_UNIT_COUNT,Étudiants_AGE_AVG,Étudiants_LAND_AREA_TOTAL,Étudiants_BUILDING_AREA_TOTAL,FIRE_RISK
0,2005,1,45.62,-73.62,0,12424,22912.0,38621.0,41.689070,4774185.0,3375618.0,373,320.0,10.0,36.528150,1157943.0,429020.0,266,501938.0,8471.0,19,886.0,73,44.0,0.0,30.808219,280703.0,101218.0,258,134.0,3.0,36.116279,368366.0,96594.0,19,1.0,0.0,3.631579,254757.0,0.0,...,1.0,27.800000,3250.0,621.0,46,57.0,9.0,43.130435,355991.0,210481.0,3,0.0,0.0,25.666667,63256.0,6022.0,2,1.0,0.0,34.00,12369.0,12007.0,2,0.0,15.000000,28972.0,2549.0,6,7.0,0.0,18.000000,12172.0,1517.0,1,4.0,5.0,9.000000,5220.0,0.0,2.0
1,2005,1,45.49,-73.58,1,21903,26711.0,54682.0,51.711409,2537161.0,16476280.0,2373,3348.0,297.0,55.383059,1706791.0,7617611.0,2009,860141.0,89618.0,2,1198.0,41,55.0,6.0,53.853659,846167.0,167834.0,198,50.0,14.0,19.530303,2464043.0,56181.0,65,6.0,0.0,19.569231,2859582.0,103429.0,...,33.0,80.300000,59627.0,14833.0,88,170.0,42.0,55.625000,511539.0,910873.0,40,26.0,4.0,56.625000,539817.0,137906.0,24,13.0,0.0,59.25,276251.0,285261.0,6,8.0,61.833333,24472.0,15553.0,46,43.0,1.0,65.695652,10946.0,89167.0,13,70.0,26.0,62.076923,48131.0,125131.0,3.0
2,2005,1,45.48,-73.69,2,18856,23343.0,35527.0,29.361423,5763462.0,9377079.0,726,723.0,5.0,29.013774,8090310.0,2719256.0,715,3894999.0,39219.0,18,5182.0,240,121.0,0.0,22.375000,3692195.0,1254622.0,579,423.0,0.0,26.350604,9890935.0,1093602.0,48,2.0,0.0,9.854167,1272345.0,1542.0,...,0.0,0.000000,0.0,0.0,59,51.0,2.0,43.559322,741242.0,186990.0,6,8.0,2.0,49.333333,80889.0,35399.0,2,3.0,0.0,37.00,4667.0,844.0,3,0.0,6.333333,29704.0,1789.0,8,8.0,0.0,23.125000,21994.0,1516.0,3,7.0,165.0,39.000000,4991.0,102.0,1.0
3,2005,1,45.54,-73.55,3,25257,44836.0,66187.0,49.637012,6780004.0,9244519.0,368,392.0,26.0,45.290761,1881853.0,1633321.0,755,1540265.0,25553.0,9,10843.0,57,67.0,0.0,45.894737,3197240.0,566205.0,95,41.0,10.0,27.863158,964932.0,66015.0,39,7.0,2.0,17.769231,1591331.0,13091.0,...,2.0,40.666667,16564.0,848.0,80,58.0,2.0,40.162500,753976.0,261331.0,15,12.0,18.0,44.000000,171880.0,28484.0,10,5.0,6.0,42.90,330657.0,131033.0,6,4.0,57.166667,860673.0,221951.0,26,25.0,0.0,35.923077,41118.0,8020.0,0,0.0,0.0,0.000000,0.0,0.0,1.0
4,2005,1,45.61,-73.63,0,12424,22912.0,38621.0,41.6

In [27]:
# Normalize the numerical data train_X

cols_to_norm = ['YEAR','MONTH','LATITUDE',
'LONGITUDE',
'NOM_ARROND',
'Résidentiel_BUILDING_COUNT',
'Résidentiel_STORY_COUNT',
'Résidentiel_UNIT_COUNT',
'Résidentiel_AGE_AVG',
'Résidentiel_LAND_AREA_TOTAL',
'Résidentiel_BUILDING_AREA_TOTAL',
'Commercial_BUILDING_COUNT',
'Commercial_STORY_COUNT',
'Commercial_UNIT_COUNT',
'Commercial_AGE_AVG',
'Commercial_LAND_AREA_TOTAL',
'Commercial_BUILDING_AREA_TOTAL',
'Terrain vague_BUILDING_COUNT',
'Terrain vague_LAND_AREA_TOTAL',
'Terrain vague_BUILDING_AREA_TOTAL',
'Ruelle_BUILDING_COUNT',
'Ruelle_LAND_AREA_TOTAL',
'Industriel_BUILDING_COUNT',
'Industriel_STORY_COUNT',
'Industriel_UNIT_COUNT',
'Industriel_AGE_AVG',
'Industriel_LAND_AREA_TOTAL',
'Industriel_BUILDING_AREA_TOTAL',
'Transport_BUILDING_COUNT',
'Transport_STORY_COUNT',
'Transport_UNIT_COUNT',
'Transport_AGE_AVG',
'Transport_LAND_AREA_TOTAL',
'Transport_BUILDING_AREA_TOTAL',
'Parc_BUILDING_COUNT',
'Parc_STORY_COUNT',
'Parc_UNIT_COUNT',
'Parc_AGE_AVG',
'Parc_LAND_AREA_TOTAL',
'Parc_BUILDING_AREA_TOTAL',
'Lieu de culte_BUILDING_COUNT',
'Lieu de culte_STORY_COUNT',
'Lieu de culte_UNIT_COUNT',
'Lieu de culte_AGE_AVG',
'Lieu de culte_LAND_AREA_TOTAL',
'Lieu de culte_BUILDING_AREA_TOTAL',
'Construction_BUILDING_COUNT',
'Construction_STORY_COUNT',
'Construction_UNIT_COUNT',
'Construction_AGE_AVG',
'Construction_LAND_AREA_TOTAL',
'Construction_BUILDING_AREA_TOTAL',
'Immeuble inutilisé_BUILDING_COUNT',
'Immeuble inutilisé_STORY_COUNT',
'Immeuble inutilisé_AGE_AVG',
'Immeuble inutilisé_LAND_AREA_TOTAL',
'Immeuble inutilisé_BUILDING_AREA_TOTAL',
'Éducation_BUILDING_COUNT',
'Éducation_STORY_COUNT',
'Éducation_UNIT_COUNT',
'Éducation_AGE_AVG',
'Éducation_LAND_AREA_TOTAL',
'Éducation_BUILDING_AREA_TOTAL',
'Loisirs_BUILDING_COUNT',
'Loisirs_STORY_COUNT',
'Loisirs_UNIT_COUNT',
'Loisirs_AGE_AVG',
'Loisirs_LAND_AREA_TOTAL',
'Loisirs_BUILDING_AREA_TOTAL',
'Santé_BUILDING_COUNT',
'Santé_STORY_COUNT',
'Santé_UNIT_COUNT',
'Santé_AGE_AVG',
'Santé_LAND_AREA_TOTAL',
'Santé_BUILDING_AREA_TOTAL',
'Sécurité publique_BUILDING_COUNT',
'Sécurité publique_STORY_COUNT',
'Sécurité publique_AGE_AVG',
'Sécurité publique_LAND_AREA_TOTAL',
'Sécurité publique_BUILDING_AREA_TOTAL',
'Restaurant_BUILDING_COUNT',
'Restaurant_STORY_COUNT',
'Restaurant_UNIT_COUNT',
'Restaurant_AGE_AVG',
'Restaurant_LAND_AREA_TOTAL',
'Restaurant_BUILDING_AREA_TOTAL',
'Étudiants_BUILDING_COUNT',
'Étudiants_STORY_COUNT',
'Étudiants_UNIT_COUNT',
'Étudiants_AGE_AVG',
'Étudiants_LAND_AREA_TOTAL',
'Étudiants_BUILDING_AREA_TOTAL',
'FIRE_RISK']
merged_features[cols_to_norm] = merged_features[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
merged_features.head(5)

,YEAR,MONTH,LATITUDE,LONGITUDE,NOM_ARROND,Résidentiel_BUILDING_COUNT,Résidentiel_STORY_COUNT,Résidentiel_UNIT_COUNT,Résidentiel_AGE_AVG,Résidentiel_LAND_AREA_TOTAL,Résidentiel_BUILDING_AREA_TOTAL,Commercial_BUILDING_COUNT,Commercial_STORY_COUNT,Commercial_UNIT_COUNT,Commercial_AGE_AVG,Commercial_LAND_AREA_TOTAL,Commercial_BUILDING_AREA_TOTAL,Terrain vague_BUILDING_COUNT,Terrain vague_LAND_AREA_TOTAL,Terrain vague_BUILDING_AREA_TOTAL,Ruelle_BUILDING_COUNT,Ruelle_LAND_AREA_TOTAL,Industriel_BUILDING_COUNT,Industriel_STORY_COUNT,Industriel_UNIT_COUNT,Industriel_AGE_AVG,Industriel_LAND_AREA_TOTAL,Industriel_BUILDING_AREA_TOTAL,Transport_BUILDING_COUNT,Transport_STORY_COUNT,Transport_UNIT_COUNT,Transport_AGE_AVG,Transport_LAND_AREA_TOTAL,Transport_BUILDING_AREA_TOTAL,Parc_BUILDING_COUNT,Parc_STORY_COUNT,Parc_UNIT_COUNT,Parc_AGE_AVG,Parc_LAND_AREA_TOTAL,Parc_BUILDING_AREA_TOTAL,...,Immeuble inutilisé_STORY_COUNT,Immeuble inutilisé_AGE_AVG,Immeuble inutilisé_LAND_AREA_TOTAL,Immeuble inutilisé_BUILDING_AREA_TOTAL,Éducation_BUILDING_COUNT,Éducation_STORY_COUNT,Éducation_UNIT_COUNT,Éducation_AGE_AVG,Éducation_LAND_AREA_TOTAL,Éducation_BUILDING_AREA_TOTAL,Loisirs_BUILDING_COUNT,Loisirs_STORY_COUNT,Loisirs_UNIT_COUNT,Loisirs_AGE_AVG,Loisirs_LAND_AREA_TOTAL,Loisirs_BUILDING_AREA_TOTAL,Santé_BUILDING_COUNT,Santé_STORY_COUNT,Santé_UNIT_COUNT,Santé_AGE_AVG,Santé_LAND_AREA_TOTAL,Santé_BUILDING_AREA_TOTAL,Sécurité publique_BUILDING_COUNT,Sécurité publique_STORY_COUNT,Sécurité publique_AGE_AVG,Sécurité publique_LAND_AREA_TOTAL,Sécurité publique_BUILDING_AREA_TOTAL,Restaurant_BUILDING_COUNT,Restaurant_STORY_COUNT,Restaurant_UNIT_COUNT,Restaurant_AGE_AVG,Restaurant_LAND_AREA_TOTAL,Restaurant_BUILDING_AREA_TOTAL,Étudiants_BUILDING_COUNT,Étudiants_STORY_COUNT,Étudiants_UNIT_COUNT,Étudiants_AGE_AVG,Étudiants_LAND_AREA_TOTAL,Étudiants_BUILDING_AREA_TOTAL,FIRE_RISK
0,0.0,0.0,0.998249,0.949275,0.000000,0.358030,0.450199,0.459063,0.506616,0.357293,0.064967,0.119246,0.093567,0.023202,0.504680,0.134335,0.056304,0.132404,0.089189,0.094523,0.431818,0.005958,0.294355,0.131343,0.000000,0.430409,0.037534,0.079935,0.418831,0.310905,0.214286,0.816122,0.034887,0.087419,0.121019,0.058824,0.000000,0.107954,0.023928,0.000000,...,0.030303,0.314836,0.002466,0.012860,0.500000,0.329480,0.163636,0.639538,0.067823,0.214604,0.071429,0.000000,0.00,0.332255,0.095272,0.043667,0.071429,0.041667,0.0,0.398827,0.019711,0.029775,0.285714,0.0,0.145161,0.033662,0.011485,0.109091,0.148936,0.0,0.107143,0.296026,0.016883,0.05,0.057143,0.016556,0.113565,0.088365,0.000000,0.5
1,0.0,0.0,0.995405,0.963768,0.035714,0.631192,0.524846,0.649970,0.628411,0.189877,0.317103,0.758632,0.978947,0.689095,0.765184,0.198008,0.999716,1.000000,0.152837,1.000000,0.045455,0.008056,0.165323,0.164179,0.096774,0.752367,0.113144,0.132543,0.321429,0.116009,1.000000,0.441328,0.233364,0.050845,0.414013,0.352941,0.000000,0.581725,0.268589,1.000000,...,1.000000,0.909400,0.045243,0.307171,0.956522,0.982659,0.763636,0.824807,0.097458,0.928716,0.952381,0.928571,0.10,0.733010,0.813036,1.000000,0.857143,0.541667,0.0,0.695015,0.440230,0.707393,0.857143,1.0,0.598387,0.028434,0.070074,0.836364,0.914894,1.0,0.391046,0.266209,0.992354,0.65,1.000000,0.086093,0.783305,0.814772,1.000000,1.0
2,0.0,0.0,0.995186,0.923913,0.071429,0.543385,0.458668,0.422287,0.356808,0.431329,0.180471,0.232097,0.211404,0.011601,0.400860,0.938571,0.356868,0.355898,0.692098,0.437624,0.409091,0.034848,0.967742,0.361194,0.000000,0.312592,0.493694,0.990810,0.939935,0.981439,0.000000,0.595447,0.936747,0.989730,0.305732,0.117647,0.000000,0.292930,0.119506,0.014909,...,0.000000,0.000000,0.000000,0.000000,0.641304,0.294798,0.036364,0.645897,0.141221,0.190653,0.142857,0.285714,0.05,0.638619,0.121830,0.256689,0.071429,0.125000,0.0,0.434018,0.007437,0.002093,0.428571,0.0,0.061290,0.034513,0.008060,0.145455,0.170213,0.0,0.137649,0.534900,0.016872,0.15,0.100000,0.546358,0.492114,0.084489,0.000815,0.0
3,0.0,0.0,0.996499,0.974638,0.107143,0.727846,0.880986,0.786723,0

In [0]:
from sklearn.model_selection import train_test_split
train_data ,test_data = train_test_split(merged_features,test_size=0.3)

In [0]:
train_X = train_data.drop(['YEAR', 'FIRE_RISK'], axis=1)
train_y = train_data[['FIRE_RISK']]

test_X = test_data.drop(['YEAR', 'FIRE_RISK'], axis=1)
test_y = test_data[['FIRE_RISK']]

In [30]:
!pip install keras-adabound

In [31]:
from keras import Sequential
from keras import layers
from keras import backend as K
from keras.layers.core import Dense
from keras import regularizers
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.constraints import max_norm
from keras_adabound import AdaBound

def create_model():
    model = Sequential()
    
    # Input layer
    model.add(Dense(units = 1000, activation = 'relu', input_shape=(91,), init='normal'))
    model.add(BatchNormalization())
    # Add dropout regularization
    model.add(Dropout(rate=0.5))

    # Hidden layer
    model.add(Dense(units = 500, activation='relu', init='normal'))
    model.add(BatchNormalization())
    # Add dropout regularization
    model.add(Dropout(rate=0.4))
    
    # Output layer
    model.add(Dense(units = 1, activation='sigmoid'))

    return model

Using TensorFlow backend.


In [32]:
model_adabound = create_model()
model_adabound.summary()

W0722 23:49:52.056572 140086362171264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1000, activation="relu", input_shape=(91,), kernel_initializer="normal")`
  from ipykernel import kernelapp as app
W0722 23:49:52.074017 140086362171264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 23:49:52.076357 140086362171264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0722 23:49:52.152761 1400863621712

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              92000     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1000)              4000      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               500500    
_________________________________________________________________
batch_normalization_2 (Batch (None, 500)               2000      
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 501       
Total para

In [33]:
model_adabound.compile(loss='binary_crossentropy', optimizer=AdaBound(lr=1e-3, final_lr=0.1), metrics=['accuracy'])

W0722 23:50:04.194485 140086362171264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0722 23:50:04.205219 140086362171264 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [34]:
train_X.shape

(66831, 91)

In [35]:
train_y.shape

(66831, 1)

In [36]:
h_web = model_adabound.fit(train_X, train_y, batch_size=10000, epochs=100, validation_split=0.3, shuffle=True)

Train on 46781 samples, validate on 20050 samples
Epoch 1/100
46781/46781 [==============================] - 2s 40us/step - loss: 1.0098 - acc: 0.4563 - val_loss: 0.8124 - val_acc: 0.5252
Epoch 2/100
46781/46781 [==============================] - 0s 6us/step - loss: 0.8106 - acc: 0.4934 - val_loss: 0.6643 - val_acc: 0.6072
Epoch 3/100
46781/46781 [==============================] - 0s 6us/step - loss: 0.7174 - acc: 0.5343 - val_loss: 0.5801 - val_acc: 0.6700
Epoch 4/100
46781/46781 [==============================] - 0s 6us/step - loss: 0.6605 - acc: 0.5709 - val_loss: 0.5608 - val_acc: 0.7104
Epoch 5/100
46781/46781 [==============================] - 0s 5us/step - loss: 0.6148 - acc: 0.6107 - val_loss: 0.5391 - val_acc: 0.6836
Epoch 6/100
46781/46781 [==============================] - 0s 5us/step - loss: 0.5746 - acc: 0.6478 - val_loss: 0.4886 - val_acc: 0.7587
Epoch 7/100
46781/46781 [==============================] - 0s 5us/step - loss: 0.5451 - acc: 0.6674 - val_loss: 0.4711 - val_ac

In [37]:
_, train_acc = model_adabound.evaluate(train_X, train_y, verbose=0)
_, test_acc = model_adabound.evaluate(test_X, test_y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.799, Test: 0.797
